# 02_create_sag_master_table

This script creates a master table for all SAGs with columns for all of the information available on google drive.

In [28]:
# Load packages
import pandas as pd
import os
import os.path as op
import sys
import csv
import numpy as np
import matplotlib
import glob
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt

# file paths
path = '/Users/melissaherring/Google Drive/My Drive/MH_project/'
#path = '/Users/juliabrown/Google Drive/My Drive/projects/OMZvir_round2/MH_project/'

# define functions 
def split_classification(df):
    df[['domain','phyla','class','order','family','genus','species']] = df.classification.str.split(';', expand=True)
    df['domain'] = df['domain'].str.replace('d__', '')
    df['phyla'] = df['phyla'].str.replace('p__', '')
    df['class'] = df['class'].str.replace('c__', '')
    df['order'] = df['order'].str.replace('o__', '')
    df['family'] = df['family'].str.replace('f__', '')
    df['genus'] = df['genus'].str.replace('g__', '')
    df['species'] = df['species'].str.replace('s__', '')
    return df

# input sag data
proximeta_matches = pd.read_csv(op.join(path, 'data/sag_data/MPsag_mx2_proximeta_matches_gtdbtk.csv'))
vsag_info = pd.read_csv(op.join(path, 'data/sag_data/MPvsag_info_230818.csv'))

# create virus_type column for vsag_info
vsag_info['virus_type'] = 'vsag'

# rename well column for proximeta_matches
proximeta_matches = proximeta_matches.rename(columns={'well': 'sag'})

# merge
sag_master = proximeta_matches.merge(vsag_info,how='outer',on=['sag','classification_via_GTDBTk','depth']).replace('', np.nan).fillna("NA").drop('plate_y',axis=1)

sag_master['virus_type'] = sag_master['virus_type'].replace('NA', 'uninfected sag')

sag_master = sag_master.rename(columns={'plate_x':'plate','classification_via_GTDBTk':'classification'})

sag_master['classification'] = sag_master['classification'].replace('0', np.nan).fillna("NA")

In [30]:
sag_master.to_csv(op.join(path, 'tables/sag_master.csv'), index=False)